In [1]:
%run -i utils.ipynb
d = load_event_log()

/home/kulla/.local/share/asdf/installs/python/3.9.1/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [38]:
users = d[(d["event_category"] == "edit") & (d["date"] > pd.to_datetime("2019-01-01"))]["user_id"].unique()
raw = d.copy()

def getdata(users):
    def get_user(user_id):
        username, last_login, registration_date = query(f"""
            select user.username, user.last_login, user.date
            from user where user.id = {user_id}
        """)[0]
        
        
        events = raw[raw["user_id"] == user_id]
        
        edits = events[events["event_category"] == "edit"].copy()
        edits.set_index("date", inplace=True)
        
        def has_role(role):
            return len(query(f"""
                select role.id
                from role
                join role_user on role_user.role_id = role.id
                join user on user.id = role_user.user_id
                where role.name = "{role}"
                and user.id = {user_id}
            """)) > 0
        
        def get_date(is_start=False, baseline=1):
            d = edits.copy()
            d = d[["user_id"]]
            d = d.resample("D").count()
            d = d.rolling(30).sum()
            d.dropna(inplace=True)
            
            dt = d[d["user_id"] >= baseline]
            
            if len(dt) == 0:
                return None
            
            if is_start:
                return dt.index[0]
            else:
                return dt.index[-1]
        
        return {
            "user_id": user_id,
            "username": username,
            "last_login": last_login,
            "registration_date": registration_date,
            "count_event_of_type_edit": len(events[events["event_category"] == "edit"]),
            "count_event_of_type_review": len(events[events["event_category"] == "review"]),
            "count_event_of_type_moderation": len(events[events["event_category"] == "moderation"]),
            "count_event_of_type_admin": len(events[events["event_category"] == "admin"]),
            "count_event_of_type_taxonomy": len(events[events["event_category"] == "taxonomy-architekt"]),
            "first_time_over_1edit": get_date(is_start=True, baseline=1),
            "first_time_over_10edit": get_date(is_start=True, baseline=10),
            "first_time_over_50edit": get_date(is_start=True, baseline=50),
            "first_time_over_100edit": get_date(is_start=True, baseline=100),
            "last_time_over_1edit": get_date(is_start=False, baseline=1),
            "last_time_over_10edit": get_date(is_start=False, baseline=10),
            "last_time_over_50edit": get_date(is_start=False, baseline=50),
            "last_time_over_100edit": get_date(is_start=False, baseline=100),
            "is_de_admin_currently": has_role("de_admin"),
            "is_de_moderator_currently": has_role("de_moderator"),
            "is_de_reviewer_currently": has_role("de_reviewer"),
            "is_de_architect_currently": has_role("de_architect"),
        }
    
    return pd.DataFrame((get_user(user_id) for user_id in users))

df = getdata(users)
df.to_csv("/tmp/overview_active_users.csv")
df.head()

,user_id,username,last_login,registration_date,count_event_of_type_edit,count_event_of_type_review,count_event_of_type_moderation,count_event_of_type_admin,count_event_of_type_taxonomy,first_time_over_1edit,...,first_time_over_50edit,first_time_over_100edit,last_time_over_1edit,last_time_over_10edit,last_time_over_50edit,last_time_over_100edit,is_de_admin_currently,is_de_moderator_currently,is_de_reviewer_currently,is_de_architect_currently
0,95850,Leo1,2019-02-14 15:19:19,2018-02-08 13:31:36,41,0,0,0,22,2018-03-09,...,NaT,NaT,2019-02-14,2019-02-03,NaT,NaT,False,False,False,False
1,95849,hernlmax,2019-02-13 21:21:44,2018-02-08 13:30:47,95,0,0,0,28,2018-03-09,...,2019-01-08,NaT,2019-02-13,2019-02-12,2019-02-03,NaT,False,False,False,False
2,95854,markus_janker,2019-02-27 13:48:55,2018-02-08 13:41:04,101,0,0,0,55,2018-03-09,...,2019-01-08,NaT,2019-02-27,2019-02-27,2019-02-06,NaT,False,False,False,False
3,10,Simon,2022-12-08 14:50:37,2014-03-01 20:36:32,249,23,12,15,67,2018-01-31,...,NaT,NaT,2022-12-08,2019-03-22,NaT,NaT,False,False,False,False
4,121732,Jonathan,2020-03-20 13:04:52,2018-10-22 07:45:05,1875,40,0,43,404,2018-11-20,...,2018-11-20,2018-11-20,2020-03-20,2020-03-20,2019-06-14,2019-06-11,False,False,False,False


In [39]:
d.to_csv("/tmp/event_log_since_2018.csv")